# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 61 new papers today
          22 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/21 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.00004


extracting tarball to tmp_2210.00004...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.00009


extracting tarball to tmp_2210.00009... done.
Retrieving document from  https://arxiv.org/e-print/2210.00028


...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.01051


extracting tarball to tmp_2210.01051...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.01101


extracting tarball to tmp_2210.01101...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.00560-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.00560) | **GLOSTAR -- Radio Source Catalog II: $28^\circ < \ell < 36^\circ$ and  $|b| < 1^\circ$, VLA B-configuration**  |
|| S. A. Dzib, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2022-10-04*|
|*Comments*| *25 pages, 15 figures, and 6 tables, Accepted for publication in the Astronomy \& Astrophysics*|
|**Abstract**| As part of the GLOSTAR survey we have used the VLA in its B-configuration to observe the part of the Galactic plane between longitudes of 28d and 36d and latitudes from -1d to +1d at the C-band (4--8 GHz). To reduce the contamination of extended sources that are not well recovered by our coverage of the (u, v)-plane we discarded short baselines that are sensitive to emission on angular scales $<4''$. The resulting radio continuum images have an angular resolution of 1.0'', and sensitivity of $\sim60 \mu$Jy~beam$^{-1}$; making it the most sensitive radio survey covering a large area of the Galactic plane with this angular resolution. An automatic source extraction algorithm was used in combination with visual inspection to identify a total of 3325 radio sources. A total of 1457 radio sources are $\geq7\sigma$ and comprise our highly reliable catalog; 72 of these are grouped as 22 fragmented sources, e.g., multiple components of an extended and resolved source. To explore the nature of the catalogued radio sources we searched for counterparts at millimeter and infrared wavelengths. Our classification attempts resulted in 93 HII region candidates, 104 radio stars, 64 planetary nebulae, while most of the remaining radio sources are suggested to be extragalactic sources. We investigated the spectral indices ($\alpha$, $S_\nu\propto\nu^\alpha$) of radio sources classified as HII region candidates and found that many have negative values. This may imply that these radio sources represent young stellar objects that are members of the star clusters around the high mass stars that excite the HII regions, but not these HII regions themselves. By comparing the peak flux densities from the GLOSTAR and CORNISH surveys we have identified 49 variable radio sources, most of them with an unknown nature. Additionally, we provide the list of 1866 radio sources detected within 5 to 7$\sigma$ levels. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.00863-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.00863) | **Test of binary spectral model on FEROS spectra. First estimation of the  fundamental parameters for HD 20784**  |
|| Mikhail Kovalev, <mark>Sarah Gebruers</mark>, Ilya Straumit |
|*Appeared on*| *2022-10-04*|
|*Comments*| *11 pages, 10 figures, submitted to MNRAS. arXiv admin note: text overlap with arXiv:2204.10680, arXiv:2209.12548*|
|**Abstract**| We analyse spectra of 26 early-type stars, with suspected binarity, using a binary spectral model adapted for high-resolution FEROS spectra. We confirm seven SB2 candidates (AE Pic, $\epsilon$ Vol, HD 20784, HD 208433, HD 43519, HD 56024, CD-73~375A) and derive mass ratios and spectroscopic parameters for them. We find good agreement with theoretical models. For slightly evolved system HD 20784 we made the first estimation of the fundamental parameters and age $\log{t}=8.5$ yr. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.00398-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.00398) | **Globular cluster metallicity distributions in the E-MOSAICS simulations**  |
|| Joel Pfeffer, et al. -- incl., <mark>J. M. Diederik Kruijssen</mark> |
|*Appeared on*| *2022-10-04*|
|*Comments*| *18 pages, 15 figures. Submitted to MNRAS*|
|**Abstract**| The metallicity distributions of globular cluster (GC) systems in galaxies are a critical test of any GC formation scenario. In this work, we investigate the predicted GC metallicity distributions of galaxies in the MOdelling Star cluster population Assembly In Cosmological Simulations within EAGLE (E-MOSAICS) simulation of a representative cosmological volume ($L = 34.4$ comoving Mpc). We find that the predicted GC metallicity distributions and median metallicities from the fiducial E-MOSAICS GC formation model agree well the observed distributions, except for galaxies with masses $M_\ast \sim 2 \times 10^{10}$ M$_\odot$, which contain an overabundance of metal-rich GCs. The predicted fraction of galaxies with bimodal GC metallicity distributions ($37 \pm 2$ per cent in total; $45 \pm 7$ per cent for $M_\ast > 10^{10.5}$ M$_\odot$) is in good agreement with observed fractions ($44^{+10}_{-9}$ per cent), as are the mean metallicities of the metal-poor and metal-rich peaks. We show that, for massive galaxies ($M_\ast > 10^{10}$ M$_\odot$), bimodal GC distributions primarily occur as a result of cluster disruption from initially-unimodal distributions, rather than as a result of cluster formation processes. Based on the distribution of field stars with GC-like abundances in the Milky Way, we suggest that the bimodal GC metallicity distribution of Milky Way GCs also occurred as a result of cluster disruption, rather than formation processes. We conclude that separate formation processes are not required to explain metal-poor and metal-rich GCs, and that GCs can be considered as the surviving analogues of young massive star clusters that are readily observed to form in the local Universe today. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.01040-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.01040) | **Achromatic design of a photonic tricoupler and phase shifter for  broadband nulling interferometry**  |
|| Teresa Klinner-Teo, et al. -- incl., <mark>Simon Gross</mark> |
|*Appeared on*| *2022-10-04*|
|*Comments*| *20 pages, 17 figures, submitted to Journal of Astronomical Telescopes, Instruments, and Systems*|
|**Abstract**| Nulling interferometry is one of the most promising technologies for imaging exoplanets within stellar habitable zones. The use of photonics for carrying out nulling interferometry enables the contrast and separation required for exoplanet detection. So far, two key issues limiting current-generation photonic nullers have been identified: phase variations and chromaticity within the beam combiner. The use of tricouplers addresses both limitations, delivering a broadband, achromatic null together with phase measurements for fringe tracking. Here, we present a derivation of the transfer matrix of the tricoupler, including its chromatic behaviour, and our 3D design of a fully symmetric tricoupler, built upon a previous design proposed for the GLINT instrument. It enables a broadband null with symmetric, baseline-phase-dependent splitting into a pair of bright channels when inputs are in anti-phase. Within some design trade space, either the science signal or the fringe tracking ability can be prioritised. We also present a tapered-waveguide $180^\circ$-phase shifter with a phase variation of $0.6^\circ$ in the $1.4-1.7~\mu$m band, producing a near-achromatic differential phase between beams{ for optimal operation of the tricoupler nulling stage}. Both devices can be integrated to deliver a deep, broadband null together with a real-time fringe phase metrology signal. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.01051-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.01051) | **Gas Morphology of Milky Way-like Galaxies in the TNG50: Signals of  Twisting and Stretching**  |
|| Thomas K. Waters, et al. -- incl., <mark>Lars Hernquist</mark>, <mark>Randall Smith</mark>, <mark>Matthew Liska</mark> |
|*Appeared on*| *2022-10-04*|
|*Comments*| *21 pages, 14 figures*|
|**Abstract**| We present an in-depth analysis of gas morphologies for a sample of 25 Milky Way-like galaxies from the Illustris TNG50 simulation. We constrain the morphology of cold, warm, hot gas, and gas particles as a whole using a Local Shell Iterative Method (LSIM) and explore its observational implications by computing the hard-to-soft X-ray ratio, which ranges between $10^{-3}$-$10^{-2}$ in the inner $\sim 50 \rm kpc$ of the distribution and $10^{-5}$-$10^{-4}$ at the outer portion of the hot gas distribution. We group galaxies into three main categories: simple, stretched, and twisted. These categories are based on the radial reorientation of the principal axes of the reduced inertia tensor. We find that the vast majority ($76\%$) of galaxies in our sample exhibit twisting patterns in their radial profiles. Additionally, we present detailed comparisons between 1) the gaseous distributions belonging to individual temperature regimes, 2) the cold gas distributions and stellar distributions, and 3) the gaseous distributions and dark matter (DM) halos. We find a strong correlation between the morphological properties of the cold gas and stellar distributions. Furthermore, we find a correlation between gaseous distributions with DM halo which increases with gas temperature, implying that we may use the gaseous morphology as a tracer to probe the DM morphology. Finally, we show gaseous distributions exhibit significantly more prolate morphologies than the stellar distributions and DM halos, which we hypothesize may be due to stellar and AGN feedback. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.01101-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.01101) | **Deep diving off the `Cosmic Cliffs': previously hidden outflows in NGC  3324 revealed by JWST**  |
|| Megan Reiter, et al. -- incl., <mark>Nathan Smith</mark>, <mark>Pamela D. Klaassen</mark> |
|*Appeared on*| *2022-10-04*|
|*Comments*| *MNRAS, accepted*|
|**Abstract**| We present a detailed analysis of the protostellar outflow activity in the massive star-forming region NGC 3324, as revealed by new Early Release Observations (ERO) from the James Webb Space Telescope (JWST). Emission from numerous outflows is revealed in narrow-band images of hydrogen Paschen-$\alpha$ (Pa-$\alpha$) and molecular hydrogen. In particular, we report the discovery of 24 previously unknown outflows based on their H$_2$ emission. We find three candidate driving sources for these H$_2$ flows in published catalogs of young stellar objects (YSOs) and we identify 15 IR point sources in the new JWST images as potential driving protostars. We also identify several Herbig-Haro (HH) objects in Pa-$\alpha$ images from JWST; most are confirmed as jets based on their proper motions measured in a comparison with previous Hubble Space Telescope (HST) H$\alpha$ images. This confirmed all previous HST-identified HH jets and candidate jets, and revealed 7 new HH objects. The unprecedented capabilities of JWST allow the direct comparison of atomic and molecular outflow components at comparable angular resolution. Future observations will allow quantitative analysis of the excitation, mass-loss rates, and velocities of these new flows. As a relatively modest region of massive star formation (larger than Orion but smaller than starburst clusters), NGC 3324 offers a preview of what star formation studies with JWST may provide. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.00341-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.00341) | **The pattern speeds of vertical breathing waves**  |
|| Tigran Khachaturyants, et al. -- incl., <mark>Soumavo Ghosh</mark> |
|*Appeared on*| *2022-10-04*|
|*Comments*| *5 pages, 4 figures, published in MNRAS Letters*|
|**Abstract**| We measure and compare the pattern speeds of vertical breathing, vertical bending, and spiral density waves in two isolated N-body+SPH simulations, using windowed Fourier transforms over 1 Gyr time intervals. We show that the pattern speeds of the breathing waves match those of the spirals but are different from those of the bending waves. We also observe matching pattern speeds between the bar and breathing waves. Our results not only strengthen the case that, throughout the disc, breathing motions are driven by spirals but indeed that the breathing motions are part and parcel of the spirals. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.00830-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.00830) | **Effects of Solar Activity, Solar Insolation and the Lower Atmospheric  Dust on the Martian Thermosphere**  |
|| N. V. Rao, et al. -- incl., <mark>V. Leelavathi</mark> |
|*Appeared on*| *2022-10-04*|
|*Comments*| *23 pages, 10 figures, 1 table, submitted for publication in JGR Planets*|
|**Abstract**| A diagnosis of the Ar densities measured by the Neutral Gas and Ion Mass Spectrometer aboard the Mars Atmosphere and Volatile EvolutioN (MAVEN) and the temperatures derived from these densities shows that solar activity, solar insolation, and the lower atmospheric dust are the dominant forcings of the Martian thermosphere. A methodology, based on multiple linear regression analysis, is developed to quantify the contributions of the dominant forcings to the densities and temperatures. The results of the present study show that a 100 sfu (solar flux units) change in the solar activity results in approx. 136 K corresponding change in the thermospheric temperatures. The solar insolation constrains the seasonal, latitudinal, and diurnal variations to be interdependent. Diurnal variation dominates the solar insolation variability, followed by the latitudinal and seasonal variations. Both the global and regional dust storms lead to considerable enhancements in the densities and temperatures of the Martian thermosphere. Using past data of the solar fluxes and the dust optical depths, the state of the Martian thermosphere is extrapolated back to Martian year (MY) 24. While the global dust storms of MY 25, MY 28 and MY 34 raise the thermospheric temperatures by approx. 22-38 K, the regional dust storm of MY 34 leads to approx. 15 K warming. Dust driven thermospheric temperatures alone can enhance the hydrogen escape fluxes by 1.67-2.14 times compared to those without the dust. Dusts effects are relatively significant for global dust storms that occur in solar minimum compared to those that occur in solar maximum. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2210.00560.md
    + _build/html/tmp_2210.00560/./HII_MB9.png
    + _build/html/tmp_2210.00560/./HII_5ghz_radio.png
    + _build/html/tmp_2210.00560/./0norelated.png
    + _build/html/tmp_2210.00560/./0resolved.png
    + _build/html/tmp_2210.00560/./0fragmented.png
    + _build/html/tmp_2210.00560/./HistoSI.png
exported in  _build/html/2210.00863.md
    + _build/html/tmp_2210.00863/./figs/HD62153_paper.png
    + _build/html/tmp_2210.00863/./figs/epsVol__paper.png
    + _build/html/tmp_2210.00863/./figs/HD20784_paper.png
    + _build/html/tmp_2210.00863/./figs/HD208433_paper.png
    + _build/html/tmp_2210.00863/./figs/HD56024_paper.png
    + _build/html/tmp_2210.00863/./figs/CCDMJ06125-6128AB_paper.png
    + _build/html/tmp_2210.00863/./figs/ltte.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\ie}{i.e., }$
$\newcommand{\lsun}{L_\odot}$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\rsun}{R_\odot}$
$\newcommand{\hi}{H~{\sc i}}$
$\newcommand{\hii}{H~{\sc ii}}$
$\newcommand{\uchii}{UC H~{\sc ii}}$
$\newcommand{\hchii}{HC H~{\sc ii}}$
$\newcommand{\sex}{\texttt{SExtractor}}$
$\newcommand{\blobcat}{\texttt{BLOBCAT}}$
$\newcommand{\rms}{r.m.s.}$
$\newcommand{\submm}{submillimetre}$
$\newcommand{\Submm}{Submillimetre}$
$\newcommand{\glo}{GLOSTAR}$
$\newcommand{\atlas}{ATLASGAL}$
$\newcommand{\fstblobs}{3880}$
$\newcommand{\Rblobs}{3325}$
$\newcommand{\Spblobs}{555}$
$\newcommand{\HRC}{1457}$
$\newcommand{\LRC}{1866}$
$\newcommand{\Plike}{904}$
$\newcommand{\Clike}{455}$
$\newcommand{\Elike}{100}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.1}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\ie$}{i.e., }$
$\newcommand{$\lsun$}{L_\odot}$
$\newcommand{$\msun$}{M_\odot}$
$\newcommand{$\rsun$}{R_\odot}$
$\newcommand{$\hi$}{H~{\sc i}}$
$\newcommand{$\hi$i}{H~{\sc ii}}$
$\newcommand{$\uchii$}{UC H~{\sc ii}}$
$\newcommand{$\hchii$}{HC H~{\sc ii}}$
$\newcommand{$\sex$}{\texttt{SExtractor}}$
$\newcommand{$\blobcat$}{\texttt{BLOBCAT}}$
$\newcommand{$\rms$}{r.m.s.}$
$\newcommand{$\submm$}{submillimetre}$
$\newcommand{$\Submm$}{Submillimetre}$
$\newcommand{$\glo$}{GLOSTAR}$
$\newcommand{$\atlas$}{ATLASGAL}$
$\newcommand{$\fstblobs$}{3880}$
$\newcommand{$\Rblobs$}{3325}$
$\newcommand{$\Spblobs$}{555}$
$\newcommand{$\HRC$}{1457}$
$\newcommand{$\LRC$}{1866}$
$\newcommand{$\Plike$}{904}$
$\newcommand{$\Clike$}{455}$
$\newcommand{$\Elike$}{100}$
$\newcommand{$\arraystretch$}{1.1}$
$\newcommand{$\arraystretch$}{1.1}$
$\newcommand{$\arraystretch$}{1.4}$
$\newcommand{$\arraystretch$}{1.4}$
$\newcommand{$\arraystretch$}{1.1}$
$\newcommand{$\arraystretch$}{1.1}$</div>



<div id="title">

# , VLA B-configuration.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.00560-b31b1b.svg)](https://arxiv.org/abs/2210.00560)<mark>Appeared on: 2022-10-04</mark> - _25 pages, 15 figures, and 6 tables, Accepted for publication in the Astronomy \& Astrophysics_

</div>
<div id="authors">

S. A. Dzib, et al. -- incl., <mark><mark>H. Beuther</mark></mark>

</div>
<div id="abstract">

**Abstract:** As part of the Global View on Star Formation (GLOSTAR) surveywe have used the Karl G. Jansky Very Large Array (VLA) in itsB-configuration to observe thepart of the Galacticplane between   longitudes of$28^\circ$and$36^\circ$and  latitudes from$-1^\circ$to$+1^\circ$at the C-band (4--8 GHz).{ To reduce the contamination ofextended sources that are not well recovered by our coverage of the($u, v$)-plane we discarded short baselines that are sensitiveto emission on angular scales $<4"$.}The resultingradio continuum images have an angular resolution of${1\rlap{.}"0}$,and a sensitivity of{${\sim60 \mu}$Jy beam$^{-1}$;}making it the mostsensitive radio survey covering a large area of the Galacticplane with this angular resolution.An automatic source extraction algorithm was used in combination withvisual inspection toidentify a total of{ $\Rblobs$}radio sources.A total of{$\HRC$}radio sources are$\geq7\sigma$and compriseour highlyreliable catalog;{72}of these are grouped as{22}fragmented sources,e.g., multiple components of an extended and resolved source.To explore the natureof the catalogued radio sources we searched for counterparts at millimeterand infrared wavelengths. Our classification attempts resulted in 93$\hi$iregion candidates,  104  radio stars,  64  planetary nebulae, while most of theremaining radio sources are suggested to be extragalactic sources.We investigated the spectral indices ($\alpha$,$S_\nu\propto\nu^{\alpha}$)of radio sources classified as$\hi$iregion candidates and found that manyhave negative values. This may imply that these radio sources represent young  stellarobjects that are members of the star clusters around the high mass stars thatexcite the$\hi$iregions, but not these$\hi$iregionsthemselves. By comparing the{peak flux densities}from the GLOSTARand CORNISH surveys we have identified{49}variable radio sources,most of them with an unknown nature.Additionally, we provide the list of{$\LRC$}radio sources detected within 5 to 7$\sigma$levels.

</div>

<div id="div_fig1">

<img src="tmp_2210.00560/./HII_MB9.png" alt="Fig11.1" width="50%"/><img src="tmp_2210.00560/./HII_5ghz_radio.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** Radio source classified as HII region. {\it Left:} Panels are from top-left to
    bottom right: RGB image of UKIDSS three observed bands, RGB image of GLIMPSE
    three observed bands, WISE 12 $\mu$m, Hi-GAL 70 $\mu$m, Hi-GAL 160 $\mu$m,
    Hi-GAL 250 $\mu$m, Hi-GAL 500 $\mu$m, and ATLASGAL 870 $\mu$m. {\it Right:}
    Radio source detected in the GLOSTAR B-configuration image. (*fig:eHII*)

</div>
<div id="div_fig2">

<img src="tmp_2210.00560/./0norelated.png" alt="Fig2.1" width="33%"/><img src="tmp_2210.00560/./0resolved.png" alt="Fig2.2" width="33%"/><img src="tmp_2210.00560/./0fragmented.png" alt="Fig2.3" width="33%"/>

**Figure 2. -** Source examples of the differences between the two sets of VLA images, D-configuration ({\it background}) and
    B-configuration ({\it contours}). Contour levels are $-5$, 5, 7, 10 and 15 times 60 $\mu$Jy beam$^{-1}$, the mean noise level of the images. Circles at the bottom-left side indicate the beam sizes of D-configuration ($18"$, white) and B-configuration ($1\rlap{.}"5$, green) images. {\it Top: } B-configuration compact radio source seen in projection in the area of a D-configuration extended radio source. {\it Middle: } D-configuration compact source that is resolved in three individual compact radio sources in the B-configuration images. {\it Bottom:} D-configuration compact source that is resolved as a fragmented radio source. The position of fragments considered in the final catalog are indicated with green crosses.   (*fig:DB*)

</div>
<div id="div_fig3">

<img src="tmp_2210.00560/./HistoSI.png" alt="Fig5" width="100%"/>

**Figure 5. -** Distribution of the in-band determined spectral indices. Blue dotted bars
    show the spectral index distribution of sources with {no counterparts} at any other
    wavelength. (*fig:hsi*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\rv}{{\rm RV}}$
$\newcommand{\ha}{\hbox{H\alpha}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\kms}{ {\rm km} {\rm s}^{-1}}$
$\newcommand{\Msun}{ {\rm M}_\odot}$
$\newcommand{\feh}{\hbox{[M/H]}}$
$\newcommand{\mgfe}{\hbox{[Mg/Fe]}}$
$\newcommand{\tife}{\hbox{[Ti/Fe]}}$
$\newcommand{\mnfe}{\hbox{[Mn/Fe]}}$
$\newcommand{\Vmic}{V_{\rm mic}}$
$\newcommand{\Vmac}{V_{\rm mac}}$
$\newcommand{\vsini}{V \sin{i}}$
$\newcommand{\logg}{\log{\rm (g)}}$
$\newcommand{\snr}{\hbox{S/N}}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\@}{tempa}$
$\newcommand{\@}{tempa }$
$\newcommand{\@}{tempb }$
$\newcommand{\@}{tempc$
$  }$
$\newcommand{\@}{tempb }$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\teff$}{T_{\rm eff}}$
$\newcommand{$\rv$}{{\rm RV}}$
$\newcommand{$\ha$}{\hbox{H\alpha}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{$\kms$}{ {\rm km} {\rm s}^{-1}}$
$\newcommand{$\Msun$}{ {\rm M}_\odot}$
$\newcommand{$\feh$}{\hbox{[M/H]}}$
$\newcommand{$\mgfe$}{\hbox{[Mg/Fe]}}$
$\newcommand{$\tife$}{\hbox{[Ti/Fe]}}$
$\newcommand{$\mnfe$}{\hbox{[Mn/Fe]}}$
$\newcommand{$\Vmic$}{V_{\rm mic}}$
$\newcommand{$\Vmac$}{V_{\rm mac}}$
$\newcommand{$\vsini$}{V \sin{i}}$
$\newcommand{$\logg$}{\log{\rm (g)}}$
$\newcommand{$\snr$}{\hbox{S/N}}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\@}{tempa}$
$\newcommand{\@}{tempa }$
$\newcommand{\@}{tempb }$
$\newcommand{\@}{tempc$
$  }$
$\newcommand{\@}{tempb }$</div>



<div id="title">

# Test of binary spectral model on FEROS spectra. First estimation of the fundamental parameters for HD 20784.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.00863-b31b1b.svg)](https://arxiv.org/abs/2210.00863)<mark>Appeared on: 2022-10-04</mark> - _11 pages, 10 figures, submitted to MNRAS. arXiv admin note: text overlap with arXiv:2204.10680, arXiv:2209.12548_

</div>
<div id="authors">

Mikhail Kovalev, <mark><mark>Sarah Gebruers</mark></mark>, Ilya Straumit

</div>
<div id="abstract">

**Abstract:** We analyse spectra of 26 early-type stars, with suspected binarity, using a binary spectral model adapted for high-resolution FEROS spectra. We confirm seven SB2 candidates (AE Pic,$\epsilon$Vol, HD 20784, HD 208433, HD 43519, HD 56024, CD-73 375A) and derive mass ratios and spectroscopic parameters for them. We find good agreement with theoretical models. For slightly evolved system HD 20784 we made the first estimation of the fundamental parameters and age$\log{t}=8.5$yr.

</div>

<div id="div_fig1">

<img src="tmp_2210.00863/./figs/HD62153_paper.png" alt="Fig7.1" width="33%"/><img src="tmp_2210.00863/./figs/epsVol__paper.png" alt="Fig7.2" width="33%"/><img src="tmp_2210.00863/./figs/HD20784_paper.png" alt="Fig7.3" width="33%"/>

**Figure 7. -** Example of the multi-epoch spectra fitting for three stars with slow rotation. The observed spectrum is shown as a gray line, the best fit is shown as a red line. The primary component is shown as the orange line, the secondary as a blue line. The difference O-C is shown as a green line. (*fig:spfit*)

</div>
<div id="div_fig2">

<img src="tmp_2210.00863/./figs/HD208433_paper.png" alt="Fig9.1" width="33%"/><img src="tmp_2210.00863/./figs/HD56024_paper.png" alt="Fig9.2" width="33%"/><img src="tmp_2210.00863/./figs/CCDMJ06125-6128AB_paper.png" alt="Fig9.3" width="33%"/>

**Figure 9. -** Same as Figure \protect\ref{fig:spfit} for three stars with a fast rotating primary and a slow rotating secondary. (*fig:spfit2*)

</div>
<div id="div_fig3">

<img src="tmp_2210.00863/./figs/ltte.png" alt="Fig5" width="100%"/>

**Figure 5. -** Folded TESS LC for HD 20784 from different sectors shown as green and red crosses respectively (top panel). $t_{min}$ folded with $P/2$ versus observation time are shown on lower panel. $t_0$ computed as median is shown as horizontal line. (*fig:period*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

54  publications in the last 7 days.
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers